In [310]:
import pandas as pd
df = pd.read_csv('movies_dataset.csv',sep=',',low_memory=False)


In [311]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

# Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- comprobando nulos


In [312]:
print(df.revenue[df.revenue.isnull()])



Series([], Name: revenue, dtype: int64)


In [313]:
print(df.budget[df.budget.isnull()])


Series([], Name: budget, dtype: int64)


- ejecutando limpieza

In [314]:
valores_a_rellenar = {'revenue':0,'budget':0}
df.fillna(value= valores_a_rellenar, inplace=True)

- comprobando que ya no hayan nulos

In [315]:
print(df.revenue[df.revenue.isnull()])
print(df.budget[df.budget.isnull()])


Series([], Name: revenue, dtype: int64)
Series([], Name: budget, dtype: int64)


In [316]:
df.budget.apply(lambda x : type(x)).value_counts()

budget
<class 'int'>    45463
Name: count, dtype: int64

## Codigo de solución:

In [317]:
valores_a_rellenar = {'revenue':0,'budget':0}
df.fillna(value= valores_a_rellenar, inplace=True)

# Los valores nulos del campo release date deben eliminarse.

- comprobando datos nulos

In [318]:
df.release_date.loc[df.release_date.isna()].shape[0]

84

- ejecutando limpieza

In [319]:
df.dropna ( axis = 0, subset = 'release_date', inplace=True)

- comprobando que haya servido la limpieza


In [320]:
df.release_date.loc[df.release_date.isna()].shape[0]

0

- solucion:

In [321]:
df.dropna ( axis = 0, subset = 'release_date', inplace=True)

# De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Solución rapida y sencilla jaja

In [322]:
# vuelvo la columna release_date al formato fecha AAAA-mm-dd
df['release_date'] = pd.to_datetime(df.release_date, format='%Y-%m-%d')
# creo la columna release_year, llenando con 0 aquellos campos donde no hay fecha registrada y lo vuelvo en un dato tipo int en caso de utilizar
# esta columna en una alguna operación matematica posterior
df['release_year'] = df.release_date.dt.year.fillna(0).astype(int)

# Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [323]:
# creo la columna return con la operación revenue/budget
df['return'] = df.revenue/ df.budget.astype(float)
# en caso de haber datos nulos, los reemplazo con cero
df['return'].fillna(0,inplace=True)
# en caso de encontrarnos con datos infinitos o no definidos ( operación dada por dividir por cero), reemplazo con 0
df['return'].replace([float('inf'), float('-inf')],0,inplace=True)

# Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.

In [324]:
df.drop(['video','imdb_id','adult','original_title','vote_count','poster_path','homepage'],axis=1,inplace=True)


- confirmo que las columnas hayan sido eliminadas

In [325]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'release_year', 'return'],
      dtype='object')

# Guardo el DF

In [326]:

df.to_csv('Clean_movies_dataset.csv')